# 병원/인구 데이터로 시군구별 의료취약도 지수를 만들어서 예측
  - 데이터 추가 수집 (open api, csv 다운로드)
  - 데이터 집계 및 병합 → 지표 생성 → 시각화 → 모델 학습/평가 → 해석 → 교차검증 → 결론
  > 이 지역이 얼마나 의료 사각지대인지 수치화하는 것이 목표

1) 데이터 수집 (open api)
  - 의료 사각지대 머신러닝 기반 예측
  - https://www.data.go.kr/data/15099158/fileData.do (지역별(법정동) 성별 연령별 주민등록 인구수)
  > csv 파일 다운로드해서 사용

2) 데이터 로드 및 정제


In [ ]:
import pandas as pd
import numpy as np

# 데이터 로드
hosp = pd.read_csv("./data/hospitals_all.csv")
people = pd.read_csv("./data/people.csv", encoding="cp949")

: 

In [ ]:
# 2. 기본 구조 확인
print("=== Hospital Data ===")
print(hosp.shape)
print(hosp.dtypes.head())
print(hosp.head())

print("\n=== People Data ===")
print(people.shape)
print(people.dtypes.head())
print(people.head())

# 3. 결측치 개요
print("\nHospital NA summary:")
print(hosp.isna().sum().sort_values(ascending=False).head(15))

print("\nPeople NA summary:")
print(people.isna().sum().sort_values(ascending=False).head(15))

# 병원 데이터 (hospital_all.csv)
- 행/열: 79,256 × 30 → 전국 단위 병원·의원·보건소 정보

## 주요 변수
- addr: 주소 (약 7만6천 개 고유값, 결측치 없음)
- clCd, clCdNm: 병원종별 코드와 명칭
- XPos, YPos: 좌표값
- ykiho: 의료기관 고유코드
- drTotCnt, pnursCnt: 전체 의사 수, 간호사 수 (결측치 없음, 분포 편차 큼)
- estbDd: 설립일 (YYYYMMDD 정수, 결측 없음)
- hospUrl: 병원 웹사이트 주소 (7만 건 이상 결측 → 분석 불필요)
- cmdc*, dety*, mdept*: 전공의·인턴·레지던트·전임의 수 (대부분 0, 소수 병원만 값 존재)
---
# 인구 데이터 (people.csv)
- 행/열: 18,649 × 231 → 행정동 단위, 월별 인구 데이터

## 주요 변수
- 시도명, 시군구명, 읍면동명, 법정동코드: 행정구역 정보
- 기준연월: 인구 집계 기준 시점 (YYYYMM)
- 계, 남자, 여자: 총인구 및 성별 인구 수
- 연령별 인구: 0세 ~ 110세 이상까지 세분화된 인구 수 (세대별/성별 구분 포함)
---
# 결측치 요약 정리

- 병원 데이터
  - hospUrl: 7만 건 이상 결측 → 분석 불필요
  - emdongNm, telno: 일부 결측
  - XPos, YPos: 7건만 결측

- 인구 데이터
  - 리명: 일부 결측 (읍·면 지역 특성)
  - 시군구명: 약 8건 결측 → 보완 가능
---
## 정리
- 병원 데이터는 지역별 의료 공급(기관·인력) 현황을 파악하는 데 충분했음
- 인구 데이터는 지역별 수요(총인구, 성별, 연령 구조, 고령자 비율) 산출에 필요할 것으로 확인
- 두 데이터셋을 시도명 / 시군구명을 기준으로 병합하여 공급-수요 결합에 대해서 만들 수 있음
- 이 결합 지표는 이후 의료 취약도 산출 및 머신러닝 기반 예측 모델링에 활용

3) 데이터 병합 및 EDA시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Windows 기준: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'

# -----------------------------
# 1. 병원 데이터 집계
# -----------------------------
hosp_group = hosp.groupby("sgguCdNm").agg({
    "ykih0": "count",   # 병원 수
    "drTotCnt": "sum",  # 의사 수 합계
    "pnursCnt": "sum"   # 간호사 수 합계
}).rename(columns={"ykih0": "hospital_count"})

# -----------------------------
# 2. 인구 데이터 집계
# -----------------------------
people_group = people.groupby("시군구명").agg({
    "계": "sum"   # 전체 인구 수
})

# 65세 이상 인구 합계 계산
# "세"와 "남/여"가 포함된 컬럼만 추출 후 65세 이상 컬럼만 선택
age_cols = [col for col in people.columns if ("세" in col and "남" in col) or ("세" in col and "여" in col)]
age_65_cols = [col for col in age_cols if any(str(i) + "세" in col for i in range(65, 111))]

# 시군구별 65세 이상 인구 총합, 전체 인구, 비율
people_group["pop_over65"] = people.groupby("시군구명")[age_65_cols].sum().sum(axis=1)
people_group["pop_total"] = people_group["계"]
people_group["pop_over65_rate"] = people_group["pop_over65"] / people_group["pop_total"]

# -----------------------------
# 3. 병원 데이터와 인구 데이터 병합
# -----------------------------
region_df = hosp_group.merge(people_group, left_index=True, right_index=True)

# -----------------------------
# 4. 파생 변수 생성
# -----------------------------
# 인구 대비 의사 수 (1명의 의사가 담당하는 평균 인구 수)
region_df["pop_per_doctor"] = region_df["pop_total"] / (region_df["drTotCnt"] + 1)

# -----------------------------
# 5. 시각화
# -----------------------------

# A. 지역별 의사 수 Top 20
region_df["drTotCnt"].sort_values(ascending=False).head(20).plot(kind="bar", figsize=(10, 5))
plt.title("지역별 의사 수 Top 20")
plt.ylabel("의사 수")
plt.show()

# B. 인구/의사 비율 상위 20개 지역
region_df["pop_per_doctor"].sort_values(ascending=False).head(20).plot(kind="barh", figsize=(8, 10))
plt.title("인구/의사 비율 Top 20 (의료취약 지역)")
plt.xlabel("인구 / 의사 수")
plt.ylabel("지역명")
plt.show()

# C. 고령자 비율 vs 의사 수 (버블 차트 형태)
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    region_df["pop_over65_rate"],   # x축: 고령자 비율
    region_df["drTotCnt"],          # y축: 의사 수
    c=region_df["pop_per_doctor"],  # 색상: 인구/의사 비율 (의료취약도 지표)
    cmap="coolwarm",                # 색상 팔레트
    s=region_df["pop_total"] / 1000, # 점 크기: 전체 인구 규모에 비례
    alpha=0.7, 
    edgecolors="k"
)

# 색상 범례 추가
plt.colorbar(scatter, label="인구/의사 비율 (높을수록 취약)")
plt.xlabel("고령자 비율")
plt.ylabel("의사 수")
plt.title("고령자 비율 vs 의사 수 (색: 의료취약도, 크기: 인구 규모)")
plt.show()


# 그래프 해석

1. 지역별 의사 수 Top 20
  - 강남구, 서초구, 송파구 등 수도권 대도시가 압도적으로 많은 의사를 보유
  - 지방 중소도시는 Top 20 내에 거의 없음
  > 의료 인력의 수도권 집중 현상이 뚜렷하게 드러남
---
2. 지역별 인구/의사 비율
  - 분포의 중심: 300~500명당 의사 1명 수준
  - 일부 지역은 1,000명당 의사 1명 이상 → 극단적인 취약 지역 존재
  > 단순히 의사 수만 보는 것보다, 인구 대비 자원이 의료 사각지대를 더 잘 설명
---
3. 고령자 비율 vs 의사 수 (시군구 단위)

- 고령자 비율이 높은 지역(0.30~0.50) → 의사 수가 매우 적음
- 의사 수가 많은 지역(수천 명 수준) → 고령자 비율이 낮음 (0.15~0.20)
- 버블의 크기를 통해 고령자 비율과 의사 수 규모를 확인
- 의료 수요(고령자 ↑)와 공급(의사 수 ↑) 간의 불일치 확인 → 전형적인 의료 사각지대 패턴을 보임

In [ ]:
# 의료취약도 지표(Composite Index) 생성
# - 인구 대비 의사 수 부족 정도 + 고령자 비율 반영
# - 두 가지 지표를 표준화하여 합산하면 지역별 의료취약도 의미

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

plt.rcParams['axes.unicode_minus'] = False  # 음수 기호 깨짐 방지

# 인구 대비 의사 수: 인구가 클수록, 의사가 적을수록 부족 정도를 나타냄
region_df["metric_doctor"] = region_df["pop_total"] / (region_df["drTotCnt"] + 1)

# 고령자 비중: 인구 중 65세 이상 고령 인구가 많은 비중
region_df["metric_over65"] = region_df["pop_over65_rate"] * 100

# 두 지표를 표준화(StandardScaler) -> score 표준화 적용
# 지표 간 단위가 다르므로 동일한 척도로 변환해 비교 가능
scaler = StandardScaler()
scaled = scaler.fit_transform(region_df[["metric_doctor", "metric_over65"]])
region_df[["scaled_doctor", "scaled_over65"]] = scaled

# 취약성 지표 계산 (Composite Index)
# 의사 부족 지수(metric_doctor) + 고령자 지수(metric_over65)를 동일 가중치(0.5, 0.5)로 반영
w1, w2 = 0.5, 0.5
region_df["vulnerability"] = (
    w1 * region_df["scaled_doctor"] +
    w2 * region_df["scaled_over65"]
)

# 의료취약도 분포 시각화
plt.figure(figsize=(6, 5))
region_df["vulnerability"].plot(kind="hist", bins=30)
plt.title("의료취약성 지표 분포 (Composite Index)")
plt.xlabel("의료취약성 지표 (높을수록 취약)")
plt.show()

# 상위 10개 취약 지역 확인
# 의료취약도가 가장 높은 상위 10개 시군구 Top 10 출력
top_10_vulnerability = region_df["vulnerability"].sort_values(ascending=False).head(10)
top_10_vulnerability.plot(kind="barh", figsize=(8, 6))
plt.title("의료취약성 지표 Top 10 지역")
plt.xlabel("의료취약성 점수")
plt.ylabel("지역명")
plt.show()

print(top_10_vulnerability)

# 의료취약도 지표 해석

## 1. 분포 (Histogram)
- 지표 분포는 좌우로 퍼져 있지만, 꼬리(긴 long-tail) 형태를 보임  
- 대부분의 시군구는 평균 부근(약 -1~+1 Z-score 범위)에 분포하여 보통 수준의 의료 접근성을 가짐  
- 일부 지역은 +2 이상으로 치솟으며 극단적으로 취약한 모습 확인  
- 이는 대다수 지역은 비교적 양호하나, 특정 농촌·산간 지역이 두드러진 사각지대임을 의미  

## 2. Top 10 취약 지역
1. 양양군: +2.21  
2. 영양군: +1.88  
3. 청송군: +1.66  
4. 봉화군: +1.46  
5. 정선군: +1.42  
6. 인제군: +1.25  
7. 영월군: +1.17  
8. 괴산군: +1.15  
9. 화천군: +1.10  
10. 장수군: +1.09  

## 3. 시사점
- **농촌형 취약**: 양양·영양·봉화·청송 등은 고령화율↑, 의료 인력 및 인프라 부족으로 구조적 취약성 확인  
- **도시형 취약**: 이번 Top 10에는 드러나지 않았으나, 일부 중소도시에서도 인구 대비 공급 불균형 가능성 존재  
- 따라서 의료 사각지대는 크게 두 가지 유형으로 구분 가능
  1. 농촌·산간형 취약: 고령화율↑ + 절대적 공급 부족  
  2. 도시형 취약: 인구 대비 공급 불균형  

## 정리
- 농촌·산간 지역의 의료 사각지대가 핵심 취약 요인임을 데이터로 확인됨  
- 정책 대응은 단순 의사 수 확충이 아니라, **고령층 집중 지역 맞춤형 대책(공공병원·순회진료 등)** 필요  
- 향후 머신러닝 기반 예측 모델에서는 **농촌형 vs 도시형 패턴을 구분**하여 분석하는 것이 중요함  

4) 모델 학습/평가 (피처, 타깃 구성 포함)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# 데이터 준비 및 의료취약도 지표

## 병원 데이터 전처리
- `hosp['estbYear']`: 설립일(`estbDd`)을 연도로 변환하여 병원 평균 설립연도로 추출  
  → 지역별 의료 인프라의 노후화 정도를 파악하기 위한 지표
- `hosp_group`: 시군구(`sgguCdNm`) 단위로 병원 수(`hospital_count`), 의사 수(`drTotCnt`), 평균 설립연도(`estbYear_mean`) 집계  
  → 지역별 기본 의료 공급 능력을 보여줌
- `type_cnt` 및 `type_share`: 시군구별 병원 유형(종합병원, 의원 등) 비율 산출  
  → 단순한 병원 수 외에도 병원 종류의 구조 차이를 반영

---

## 인구 데이터 전처리
- `age65_cols`: 65세 이상 인구에 해당하는 컬럼 추출
- `p_group['pop_total']`: 시군구별 전체 인구
- `p_group['pop_over65']`: 시군구별 65세 이상 인구 합
- `p_group['pop_over65_rate']`: 고령자 비율(%)

👉 이를 통해 지역별 의료 수요 입력(고령화율) 반영

---

## 병합 및 취약도 지표 생성
- `region_df`: 병원 데이터(`hosp_group`, `type_share`)와 인구 데이터(`p_group`)를 병합
- `vulnerability`:
  - 인구 대비 의사 부족 정도 (총인구 ÷ 의사 수)
  - 고령자 비율(%)
- 두 값을 동일 가중치(50:50)로 결합  
  👉 값이 클수록 해당 지역은 의료 취약하다고 해석

---

## 추가 파생 변수
- `estbYear_mean`: 병원 평균 설립연도 → 의료 인프라의 노후화 정도 반영
- `dr_per_hosp`: 병원당 의사 수 → 효율성을 나타냄  

👉 단순 절대 규모뿐 아니라 의료 인프라의 질적 공급 효율성까지 반영

In [ ]:
# hosp, people, region_df, vulnerability 가 이전 단계에서 존재한다고 가정
# region_dt 에 최소 다음 컬럼이 있어야 함:
#   ['hospital_count', 'drTotCnt', 'pop_total', 'pop_over65_rate', 'vulnerability']

if region_df not in globals():
    # 1) 병원 집계
    hosp2 = hosp.copy()
    hosp2['estbYear'] = (hosp2['estbYmd'] // 10000).astype(int)
    hosp_group = hosp2.groupby("sgguCdNm").agg(
        hospital_count=("ykih0", "count"),
        drTotCnt=("drTotCnt", "sum"),
        estbYear_mean=("estbYear", "mean")
    )

    # 2) 병원 유형 비율
    type_cnt = hosp2.pivot_table(index="sgguCdNm", columns="clCdNm", values="ykih0", aggfunc="count", fill_value=0)
    type_share = type_cnt.div(type_cnt.sum(axis=1).replace(0, np.nan), axis=0).add_prefix("share_").fillna(0)

    # 3) 인구 집계
    age_cols = [c for c in people.columns if ('세' in c and ('남' in c or '여' in c))]
    age65_cols = [c for c in age_cols if any(str(i) + '세' in c for i in range(65, 111))]

    p_group = people.groupby("시군구명").agg({'계': 'sum'})
    p_group['pop_total'] = p_group['계']
    p_group['pop_over65'] = people.groupby("시군구명")[age65_cols].sum().sum(axis=1)
    p_group['pop_over65_rate'] = (p_group['pop_over65'] / p_group['pop_total']).replace([np.inf, -np.inf], np.nan).fillna(0)

    region_df = hosp_group.join(type_share, how="inner").join(p_group, how="inner")

    # 4) 취약도(50:50 가중치 - 의사와 고령자 비율)
    w1, w2 = 0.5, 0.5
    region_df['vulnerability'] = (
        w1 * (region_df['pop_total'] / (region_df['drTotCnt'] + 1)) +
        w2 * (region_df['pop_over65_rate'] * 100)
    )

# 추가 파생 변수 생성
if 'estbYear_mean' not in region_df.columns and 'estbDd' in hosp.columns:
    tmp = hosp.copy()
    tmp['estbYear'] = (tmp['estbDd'] // 10000).astype(int)
    estb = tmp.groupby('sgguCdNm')['estbYear'].mean().rename('estbYear_mean')
    region_df = region_df.join(estb, how='left')

# 결측 보정
region_df = region_df.replace([np.inf, -np.inf], np.nan)

# 병원 유형 비율 컬럼 목록
share_cols = [c for c in region_df.columns if c.startswith('share_')]

# 비율/혼합 파생
region_df['dr_per_hosp'] = region_df['drTotCnt'] / region_df['hospital_count'].replace(0, np.nan)


# 의료취약도 예측 데이터 준비

## 목적
- 시군구 단위의 의료 취약도(`vulnerability`)를 예측하기 위해,  
  인구·고령화율(수요), 병원 수·의사 수(공급) 등 주요 지표를 바탕으로 여러 단계의 피처셋(feature set)을 생성  
- 이를 안전하게 학습 데이터(X, y)로 변환

---

## Feature set
- `X_base_cols`: 총인구, 65세 이상 비율, 병원 수, 병원 평균 설립연도, 병원 유형 비율
- `X_plus_cols`: 위 항목 + 병원당 의사 수(공급 밀도)
- `X_full_cols`: 위 합 + 전체 의사 수(절대 공급 규모)

---

## 데이터 준비 방식
- `build_Xy(cols)` 함수로 피처셋과 타깃(`vulnerability`) 생성
- 주요 단계:
  1. 컬럼 존재 여부 확인
  2. 결측치 및 무한값 제거
  3. 깨끗한 `X`, `y` 반환

In [ ]:
# 피처셋 정의 (의사수/고령자 중심) ========

# 기본 피처: 수요/공급/지역 특성
x_base_cols = [
    "pop_total",        # 총인구 (수요)
    "pop_over65_rate",  # 65세 이상 비율 (수요 압박)
    "hospital_count",   # 시군구 병원 수 (공급)
    "drTotCnt",         # 시군구 전체 의사 수 (공급)
    "estbYear_mean",    # 병원 평균 설립연도 (참조: 인프라 노후도)
] + share_cols          # 병원 유형 비율(share_*)

# 확장 피처: 병원당 의사 수 등 공급 밀도 정보 추가
x_plus_cols = x_base_cols + [
    "dr_per_hosp"       # 병원당 의사 수 (공급 밀도)
]

# 전체 피처: 절대 의사 수 포함 (간호사 항목 제외)
x_full_cols = x_plus_cols + [
    "drTotCnt"          # 시군구 전체 의사 수 (절대 입력)
    # pnursCnt 제외: 현재 pnursCnt는 조산사수라, 전체 간호사 수가 아님 -> 제외
]

# 입력값 X,y 생성 함수: 결측치(무한대, NaN) 제거 처리 포함
def build_xy(cols):

    # 지정된 피처셋이 region_df에 모두 존재하는지 확인
    missing = [c for c in cols if c not in region_df.columns]
    if missing:
        raise KeyError(f"region_df에 존재하지 않는 컬럼이 있습니다: {missing}")

    X = region_df[cols].copy()
    y = region_df["vulnerability"].copy()

    # 결측치 결합 제거 (X 또는 y에 결측치 있으면 행 삭제)
    df_xy = pd.concat([X, y], axis=1)
    df_xy = df_xy.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any")

    X_clean = df_xy[cols]
    y_clean = df_xy["vulnerability"]

    return X_clean, y_clean

# 목적
의료 취약도 예측을 위해 여러 피쳐셋(`X_base`, `X_plus`, `X_full`)과 여러 모델(선형회귀, 랜덤포레스트)을 동일한 전처리 흐름에서 평가하여 어떤 조합이 가장 좋은지 비교

---

## 전처리 파이프라인
- 결측치 처리: `SimpleImputer(strategy='median')`
- 스케일링: `StandardScaler()`
- 모델 학습기: 선택된 회귀 모델 (`LinearRegression` or `RandomForestRegressor`)

---

## 실험 구성

### 모델 종류
- `LinearRegression`
- `RandomForestRegressor` (500트리)

### 피쳐셋 종류
- `X_base` : 기본 지표 (인구-고령화율, 병원 수 등)
- `X_plus` : + 병원당 의사 수
- `X_full` : + 전체 의사 수

### 타깃 종류
- 원본 `y`

---

총 **3(피쳐셋) × 2(모델) × 1(타깃) = 6가지 조합**을 평가한다.

In [ ]:
def make_pipe(model):
    num_cols = None
    pre = ColumnTransformer(
        transformers=[
            ('imp', Pipeline(steps=[
                ('imp', SimpleImputer(strategy='median')),
                ('sc', StandardScaler())
            ]), list(range(0))) # dummy; 아래 파이프에서 바로 처리
        ],
        remainder='passthrough'
    )

    # ColumnTransformer가 필요없으므로 간단히 수치 파이프만 사용
    pipe = Pipeline(steps=[
        ('imp', SimpleImputer(strategy='median')),
        ('sc', StandardScaler()),
        ('mdl', model)
    ])
    return pipe


def evaluate(model_name, X, y, log_target=False, random_state=42):
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=random_state)

    if log_target:
        ytr_t = np.log1p(ytr)
        mdl = make_pipe(LinearRegression() if model_name=='LR' else RandomForestRegressor(
            n_estimators=500, max_depth=None, random_state=random_state, n_jobs=-1
        ))
        mdl.fit(Xtr, ytr_t)
        pred_t = mdl.predict(Xte)
        yhat = np.expm1(pred_t)
    else:
        mdl = make_pipe(LinearRegression() if model_name=='LR' else RandomForestRegressor(
            n_estimators=500, max_depth=None, random_state=random_state, n_jobs=-1
        ))
        mdl.fit(Xtr, ytr)
        yhat = mdl.predict(Xte)

    mae = mean_absolute_error(yte, yhat)
    mse = mean_squared_error(yte, yhat)
    r2 = r2_score(yte, yhat)
    return {'MAE': mae, 'MSE': mse, 'R2': r2}, mdl, (Xte, yte, yhat)


# ===== 모델 실험 실행 =====
experiments = []
models = [('LR', 'LinearRegression'), ('RF', 'RandomForest')]
feature_sets = [
    ('X_base', x_base_cols),
    ('X_plus', x_plus_cols),
    ('X_full', x_full_cols),
]

for f_name, cols in feature_sets:
    X, y = build_xy(cols)
    for m_code, m_name in models:
        s1, mdl, pack1 = evaluate(m_code, X, y, log_target=False)
        s1.update({'Features': f_name, 'Model': m_name, 'Target': 'y'})
        experiments.append(s1)

results = pd.DataFrame(experiments).sort_values(['MSE','MAE']).reset_index(drop=True)
display(results)
print("==== 성능 요약 (낮을수록 좋은 항목: MAE/MSE, 높을수록 좋은 항목: R2) ====")

앞서 선택된 최적 피처셋 + 모델 + 타깃 변환 조합을 사용하여 모델을 다시 학습하고, 이후 예측 결과와 잔차(residual) 시각화를 위한 데이터를 준비

In [ ]:
# 다시 학습하여 예측/잔차 시각화 준비
X_best, y = build_xy({'X_base':X_base_cols,'X_plus':X_plus_cols,'X_full':X_full_cols}[best_feat])
use_log = (best_target=='log(y)')
code = 'LR' if best_model=='LinearRegression' else 'RF'
score, mdl, (Xte, yte, yhat) = evaluate(code, X_best, y, log_target=use_log)

# 목적
선택된 회귀 모델의 성능을 직관적으로 확인하고, 예측 결과와 오차(잔차)를 분석하며, 어떤 피처가 중요한지 해석한다.

---

## 1. 실제 vs 예측 산점도
- `X축`: 실제값, `Y축`: 예측값
- 점들이 대각선(45°)에 가까울수록 예측 성능이 우수함
- 모델의 전반적인 적합도를 직관적으로 확인할 수 있음

---

## 2. 잔차 분포
- `잔차 = 예측값 - 실제값`
- 히스토그램으로 분포를 확인하여 과대/과소 예측 경향을 파악
- 특정 구간에 편향이 있는지, 이상치가 존재하는지 점검 가능

In [ ]:
# 산점도: 실제 vs 예측
plt.figure(figsize=(6,6))
plt.scatter(yte, yhat, alpha=0.7)
plt.xlabel("실제 Vulnerability")
plt.ylabel("예측 Vulnerability")
plt.title(f"Pred vs True - {best_model} / {best_feat} / {best_target}")
lim = [0, max(yte.max(), yhat.max())*1.05]
plt.plot(lim, lim)
plt.xlim(lim); plt.ylim(lim)
plt.tight_layout(); plt.show()

# 잔차 분포
resid = yhat - yte
plt.figure(figsize=(6,4))
plt.hist(resid, bins=30)
plt.title("Residual Distribution")
plt.show()

1. 데이터셋
- `X_full_cols` (총인구, 고령자 비율, 병원 수, 의사 수 등 포함)
- `y: vulnerability` (의료 취약도 지수)

---

2. 전처리 + 모델 파이프라인
- 결측치 처리: `SimpleImputer(strategy='median')`
- 스케일링: `StandardScaler()`
- 모델: `RandomForestRegressor(random_state=42, n_jobs=-1)`
- 모든 단계를 하나의 `Pipeline`으로 연결해 일관된 학습/검증 수행

---

3. 탐색할 하이퍼파라미터 공간
- `n_estimators`: [200, 400, 600, 800, 1000]  
  → 트리 개수
- `max_depth`: [None, 10, 20, 30, 50]  
  → 트리 깊이 제한
- `min_samples_split`: [2, 5, 10, 20]  
  → 노드 분할 최소 샘플 수
- `min_samples_leaf`: [1, 2, 5, 10]  
  → 리프 노드 최소 샘플 수
- `max_features`: ['sqrt', 'log2']  
  → 분할 시 고려할 피처 수

---

4. RandomizedSearchCV 설정
- `n_iter=30`: 임의 조합 30개 시도
- `cv=5`: 5-Fold 교차검증
- `scoring='neg_root_mean_squared_error'`  
  → RMSE 기준 성능 비교
- `n_jobs=-1`: 병렬 처리

---

5. 결과 출력
- `search.best_params_` → 최적 하이퍼파라미터 조합
- `search.best_score_` → 교차검증에서의 최고 성능 (RMSE)

---

### 기대 효과
랜덤 포레스트의 기본 설정보다 **더 적합한 파라미터 조합**을 찾아  
예측 정확도와 일반화 성능을 개선

In [ ]:
# 하이퍼 파라미터 튜닝
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

# ===== 데이터셋 (최적 Feature Set: X_full) =====
X = region_df[X_full_cols]
y = region_df['vulnerability']

# ===== Pipeline (전처리 + RF) =====
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('sc', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# ===== 파라미터 공간 =====
param_dist = {
    'rf__n_estimators': [200, 400, 600, 800, 1000],
    'rf__max_depth': [None, 10, 20, 30, 50],
    'rf__min_samples_split': [2, 5, 10, 20],
    'rf__min_samples_leaf': [1, 2, 5, 10],
    'rf__max_features': ['sqrt', 'log2']
}

# ===== RandomizedSearch =====
search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=30,                     # 시도 횟수
    cv=5,                          # 5-Fold CV
    scoring='neg_root_mean_squared_error',
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X, y)

print("Best Params:", search.best_params_)
print("Best CV Score (MSE):", -search.best_score_)

## RandomForest 하이퍼파라미터 튜닝 결과와 해석

### 1. 최적 파라미터 (Best Params)
- `n_estimators = 400`
  - 총 400개의 결정 트리를 사용하여 안정적인 예측 성능 확보
- `max_depth = 50`
  - 트리 최대 깊이를 50으로 제한하여 과적합을 방지하면서 충분히 복잡한 구조 유지
- `min_samples_split = 2, min_samples_leaf = 1`
  - 최소 분할 및 leaf 크기를 가장 작게 설정하여 데이터 패턴을 세밀하게 학습
- `max_features = log2`
  - 각 분할 시 전체 피처 중 log2(n_features) 개만 랜덤하게 선택
  - 랜덤성을 높여 과적합을 줄이고 일반화 성능 개선

---

### 2. 최적 성능 (Best CV Score)
- 교차검증 기준 `RMSE = 0.2831`
- 모델 예측값이 실제값과 평균적으로 약 ±0.28 정도 차이가 있음을 의미

---

### 3. 해석
- RandomForest는 비선형 관계와 변수 간 상호작용을 잘 포착하여 선형회귀보다 유연한 성능을 보임
- `max_features = log2` 가 선택된 것은 특정 소수의 변수만 분할에 활용하는 것이 일반화에 유리함을 시사
- 모델의 일부 과적합 경향을 보일 수 있으나, 깊이 제한과 피처 샘플링 전략을 통해 균형 잡힌 결과를 얻음

---

### 4. 종합 결론
- **400개의 트리, 깊이 50, log2 피처 분할 전략**을 가진 RandomForest가 최적 조합으로 선택됨
- 성능 지표(RMSE=0.28)는 해당 데이터셋에서 상당히 안정적이고 신뢰할 수 있는 예측력을 보여줌
- 단순 선형모델 대비, 복잡한 변수 관계까지 포착할 수 있어 실무 적용 가능성이 높음
## RandomForest vs XGBoost 성능 비교

### 목적
- 동일한 피처셋(X_full)과 동일한 데이터셋에서 튜닝된 **RandomForest와 XGBoost**의 성능을 비교
- 두 모델은 모두 앙상블 기반이지만, 작동 방식과 강점이 다름

---

### 1. 모델 특성
#### RandomForest
- 배깅(Bagging) 기반: 여러 개의 결정트리를 무작위 샘플로 학습 → 평균
- 장점: 안정적, 과적합에 강함, 해석이 비교적 직관적(Feature Importance)
- 단점: 심층적 복잡 패턴 학습은 제한적일 수 있음

#### XGBoost
- 부스팅(Boosting) 기반: 이전 트리의 오차를 줄여가는 방식
- 장점: 비선형성과 상호작용에 강함, 성능이 우수한 경우가 많음
- 단점: 하이퍼파라미터 튜닝 필요, 해석이 다소 복잡

---

### 2. 실험 구성
- 데이터 분할: 80% 학습 / 20% 테스트

#### RandomForest (튜닝된 설정)
- `n_estimators=800, max_depth=10, max_features='sqrt'`

#### XGBoost
- `n_estimators=800, max_depth=6, learning_rate=0.05`
- `subsample=0.8, colsample_bytree=0.8` → 과적합 방지

---

### 4. 시각화
- 실제값 vs 예측값 산점도
  - RandomForest와 XGBoost를 나란히 비교
  - 대각선(y=x) 근처에 점이 많을수록 예측 성능이 우수함

---

### 5. 해석 포인트
#### RandomForest
- 안정적이고 해석이 직관적
- 다만 비선형 패턴이나 극단값 처리에는 한계 가능

#### XGBoost
- 복잡한 관계(비선형성, 상호작용)를 더 잘 포착
- 일반적으로 RMSE, R²에서 더 나은 성능 기대 가능

---

### 결론
- 성능 차이가 크지 않다면, **해석 용이성(RandomForest)** vs **성능(XGBoost)** 중 목적에 따라 선택

# RandomForest (tuned) vs XGBoost 성능 비교

## 1. 성능 지표
| Model                | MAE   | MSE   | R²    |
|-----------------------|-------|-------|-------|
| RandomForest (tuned)  | 0.242 | 0.114 | 0.795 |
| XGBoost              | 0.215 | 0.095 | 0.830 |

- **MAE**: XGBoost가 0.215로 더 낮아, 평균 오차가 더 작음  
- **MSE**: XGBoost가 0.095로 더 낮아, 큰 오차(Outlier)에 더 강함  
- **R²**: XGBoost가 0.83으로, 설명력(예측 성능)이 RandomForest(0.80)보다 더 우수  

---

## 2. 실제 vs 예측 산점도
- **RandomForest**:  
  - 전반적으로 대각선(y=x)에 가깝게 분포  
  - 다만 일부 음수 구간에서 과소/과대 예측이 보임  

- **XGBoost**:  
  - 더 균형 잡힌 분포  
  - 극단적인 outlier 영향이 줄어들고, 전체적으로 대각선에 더 가까움  

---

## 3. 종합 해석
- XGBoost가 MAE, MSE, R² 모든 지표에서 RandomForest보다 우수  
- RandomForest: 안정적인 예측력을 보이지만, 데이터 특성상 비선형 패턴을 더 잘 학습하는 XGBoost가 유리  
- 특히 오차 분포가 줄어들면서, 일반화 성능과 일관성 능력에서 XGBoost가 더 적합한 모델로 판단됨  

# 교차 검증
## 목적
- 단일 학습/테스트 분할에 의존하지 않고, 교차검증(5-Fold CV)으로 RandomForest와 XGBoost의 성능을 반복 측정하여 평균 성능과 변동성(표준편차)을 확인
- 이를 통해 모델의 일관성, 일반화 성능 안정성을 평가 가능

In [ ]:
# ============================
# 교차검증 안정성 평가
# ============================
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# ===== 데이터 =====
X = region_df[X_full_cols]
y = region_df['vulnerability']

# ===== 모델 정의 =====
rf_best = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('sc', StandardScaler()),
    ('rf', RandomForestRegressor(
        n_estimators=400,
        max_depth=50,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='log2',
        random_state=42,
        n_jobs=-1
    ))
])

xgb_best = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('sc', StandardScaler()),
    ('xgb', XGBRegressor(
        n_estimators=800,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        objective='reg:squarederror'
    ))
])

# ===== CV 설정 =====
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# ===== 커스텀 스코어 =====
scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'RMSE': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
    'R2': make_scorer(r2_score)
}

def evaluate_cv(model, X, y, cv):
    scores = {}
    for name, scorer in scoring.items():
        cv_scores = cross_val_score(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)
        # 부호 처리 (MAE, RMSE는 neg 값 반환 -> 붙여서 양수화)
        scores[name+'_mean'] = np.mean(-cv_scores if name in ['MAE','RMSE'] else cv_scores)
        scores[name+'_std']  = np.std(-cv_scores if name in ['MAE','RMSE'] else cv_scores)
    return scores

# ===== 실행 =====
rf_cv  = evaluate_cv(rf_best, X, y, cv)
xgb_cv = evaluate_cv(xgb_best, X, y, cv)

results_cv = pd.DataFrame([
    {'Model': 'RandomForest (tuned)', **rf_cv},
    {'Model': 'XGBoost', **xgb_cv}
])

print("=== 교차검증 성능 비교 (5-Fold) ===")
print(results_cv)

# 결과 해석

## 1. MAE (Mean Absolute Error)
- **RandomForest (tuned):** 약 0.208 (±0.049)  
- **XGBoost:** 약 0.200 (±0.043)  

두 모델 모두 낮은 수준의 평균 절대 오차를 보이지만, XGBoost가 더 작음 → **예측 정확도가 더 높음**

---

## 2. RMSE (Root Mean Squared Error)
- 현재 출력에서는 NaN으로 표시됨 → RMSE 계산이 누락된 상태  
- 추정컨대 **XGBoost가 RandomForest보다 낮을 가능성이 큼**  
  → 큰 오차(outlier)에 대한 강건성이 더 좋음  

---

## 3. R² (결정계수)
- **RandomForest (tuned):** 0.872 (±0.057)  
- **XGBoost:** 0.891 (±0.038)  

두 모델 모두 데이터 분산을 잘 설명하지만, **XGBoost가 더 높은 설명력을 가짐**  
표준편차 역시 더 작아 → 일관성이 우수함  

---

## 4. 종합 해석
- **XGBoost > RandomForest**: 평균 성능(MAE, R² 모두)에서 우수  
- 또한 작은 표준편차 값 → 교차검증 반복에서도 더 안정적인 성능을 보임  
- RandomForest: 해석 및 용이성은 직관적이나 여전히 한계가 있으며, 성능은 한 단계 낮음  

---

## 5. 결론
- **최적 모델 후보는 XGBoost**  
  → 더 높은 설명력(R²), 더 낮은 오차(MAE), 더 안정적인 결과(낮은 표준편차)

# XGBoost Feature Importance 해석

## 목적
최종 학습된 XGBoost 모델을 해석하기 위해
- Feature Importance로 변수의 상대적 중요도 확인

---

## Feature Importance (Top 15)
- `xgb_final.feature_importances_` 기반  
  → 모델이 예측할 때 가장 많이 활용한 변수 순위  
- 시각화: 상위 15개 변수의 중요도를 막대그래프로 표시  

> 의미: 모델이 어떤 변수를 주로 활용했는지를 알 수 있음 (방향성은 알 수 없음 → SHAP을 활용하면 가능하나 생략)  

단순 성능 비교를 넘어, 해석 가능한 모델로 활용하기 위함

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ===== 최종 학습 (X_full + XGBoost) =====
X = region_df[X_full_cols]
y = region_df['vulnerability']

xgb_final = XGBRegressor(
    n_estimators=800,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    objective='reg:squarederror'
)

xgb_final.fit(X, y)

# ===== Feature Importance =====
importances = xgb_final.feature_importances_
imp_df = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values('importance', ascending=False)

print("=== XGBoost Feature Importance ===")
print(imp_df.head(15))

plt.figure(figsize=(8,6))
plt.barh(imp_df['feature'][::-1], imp_df['importance'][::-1])
plt.title("XGBoost Feature Importance")
plt.xlabel("Importance")
plt.show()

# 해석 결과

## 1. 전체 변수 중요도 (bar plot)
- **drTotCnt (전체 의사 수)** 가 가장 높은 중요도 (0.44)를 차지 → 의료 취약도 예측에서 핵심 변수  
- 이어서 **dr_per_hosp (병원당 의사 수) (0.22)** 와 **hospital_count (병원 수) (0.22)** 가 비슷한 수준으로 중요하게 작용  
- 그 외 **pop_over65_rate (고령자 비율), pop_total (총인구), estbYear_mean (평균 설립연도)** 는 상대적으로 낮은 기여도를 보임  

## 2. 해석
- **핵심 변수**: 단순히 인구(pop_total)가 아니라, 의사 수(drTotCnt)와 병원당 의사 수, 병원 수 같은 공급 밀도 지표가 더 중요하게 나타남  
- **보조 변수**:
  - 고령자 비율(pop_over65_rate)은 일정 부분 영향(약 5%)을 미침  
  - 병원 설립연도(estbYear_mean)는 낮지만, 신식 병원이 많은 경우 취약도를 낮추는 방향으로 기여 가능  

## 3. 시사점
- **의료 인력 절대 규모(drTotCnt)** 확보가 취약도 개선의 가장 중요한 요소  
- 다만 인력 총량뿐만 아니라, 병원 분포(병원 수)와 인력 배치 효율(병원당 의사 수)가 함께 고려되어야 함  
- 고령화(pop_over65_rate)는 보조적 요인이지만, 특정 지역의 취약도 상승을 예측하는 데 의미 있는 지표로 작용  
- 따라서 단순 의사 수 확대 정책보다는, **지역별 병원 수와 병원당 인력 밀도 강화**가 핵심 전략이 될 수 있음  

# 병원/인구 데이터로 시군구별 의료취약도 지수를 만들어서 예측

- 데이터 추가 수집 (open api csv 다운로드)  
- 데이터 집계 및 병합 → 지표 생성 → 시각화 → 모델 학습/평가 → 해석 → 교차검증 → 결론  

> 이 지역이 얼마나 의료 사각지대인지를 수치화하는 것이 목표  

---

## 1) 데이터 수집 (open api)

의료 사각지대 머신러닝 기반 예측  

- [https://www.data.go.kr/data/15099158/fileData.do](https://www.data.go.kr/data/15099158/fileData.do) (지역별(법정동) 성별 연령별 주민등록 인구수)  
- csv 파일 다운로드해서 사용  

---

## 2) 데이터 로드 및 정제
